## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

In [37]:
import sqlite3
import nltk
import random
import numpy as np
import pandas as pd

from collections import Counter, defaultdict

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

### Import Additional Libraries and Functions

In [38]:
import string
import html
import regex as re

from nltk.corpus import stopwords
from string import punctuation

### Connect to Data

In [39]:
convention_db = sqlite3.connect("C:/Users/drzag/pol_class/2020_Conventions.db")
convention_cur = convention_db.cursor()
convention_cur

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [40]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

#query_results = convention_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

#query_results=convention_cur.execute('SELECT * FROM conventions')
convention_cur.execute('SELECT * FROM conventions')
query_results=convention_cur.fetchall()

### Get Data Structure and Column Names

result = convention_cur.execute("PRAGMA table_info('%s')" % 'conventions').fetchall()
print(result)
column_names = list(zip(*result))[1]
print (("\ncolumn names for %s:" % 'conventions'))
print(column_names)

### Use Column Names and Import Data to a List then a DataFrame for Easier Editing, then back to a list

for row in query_results:
    # store the results in convention_data
    convention_data.append(list(row))
    #pass # remove this
convention_data_db = pd.DataFrame(convention_data, columns = ['party', 'night', 'speaker', 'speaker_count', 
                                                           'time', 'text', 'text_len', 'file'])
convention_data_db2 = pd.DataFrame(convention_data_db, columns = ['text', 'party'])
convention_data = convention_data_db2.values.tolist()

### Apply Basic Tokenization and Cleaning using Pythonic list Comprehension

convention_data = [[t.translate(str.maketrans('', '', string.punctuation)),p] for t, p in convention_data]
convention_data = [[t.lower(),p] for t, p in convention_data]
convention_data[0:5]

[(0, 'party', 'TEXT', 0, None, 0), (1, 'night', 'INTEGER', 0, None, 0), (2, 'speaker', 'TEXT', 0, None, 0), (3, 'speaker_count', 'INTEGER', 0, None, 0), (4, 'time', 'TEXT', 0, None, 0), (5, 'text', 'TEXT', 0, None, 0), (6, 'text_len', 'TEXT', 0, None, 0), (7, 'file', 'TEXT', 0, None, 0)]

column names for conventions:
('party', 'night', 'speaker', 'speaker_count', 'time', 'text', 'text_len', 'file')


[['skip to content the company careers press freelancers blog × services transcription captions foreign subtitles translation freelancers about contact login « return to transcript library home  transcript categories  all transcripts 2020 election transcripts classic speech transcripts congressional testimony  hearing transcripts debate transcripts donald trump transcripts entertainment transcripts financial transcripts interview transcripts political transcripts press conference transcripts speech transcripts sports transcripts technology transcripts aug 21 2020 2020 democratic national convention dnc night 4 transcript rev  ›  blog  ›  transcripts  › 2020 election transcripts  ›  2020 democratic national convention dnc night 4 transcript night 4 of the 2020 democratic national convention dnc on august 20 read the full transcript of the event here transcribe your own content  try rev for free  and save time transcribing captioning and subtitling',
  'Democratic'],
 ['i’m here by calli

Let's look at some random entries and see if they look right. 

In [41]:
random.choices(convention_data,k=10)

[['congratulations you’re now citizens of the united states of america on behalf of the department of homeland security it is my honor to call you my fellow americans mr president i want to again commend you for your dedication to the rule of law and for restoring integrity to our immigration system thank you for hosting such a patriotic celebration here at the white house today',
  'Republican'],
 ['i’m pleased to announce that vice president joe biden has officially been nominated by the democratic party as our candidate for president of the united states',
  'Democratic'],
 ['every generation before us has had to fight for what they believe in and it’s just our turn now jack g  010203   i was proud when i saw the demonstrations that were going on across the country',
  'Democratic'],
 ['joe biden has selected kamala harris as his running mate', 'Democratic'],
 ['american history tells us that’s been in our darkest moments that we’ve made our greatest progress that we found the light

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [42]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]
print(tokens[0:5])

word_dist = nltk.FreqDist(tokens)

#for w, c in word_dist.items():
#    print(w, c)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} features in the model.")

['skip', 'to', 'content', 'the', 'company']
With a word cutoff of 5, we have 2514 features in the model.


In [43]:
#def conv_features(text,fw) :
"""
Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}  
"""

# Your code here
# ret_dict = dict()
# tokenize each piece of text into text_tokens
# for token in text_tokens if in fw:
# add token and True to a dictionary
# return(ret_dict)

sw = stopwords.words(["english"])
def conv_features(text,fw):
    #ret_list = []
    ret_dict = dict()
    #text = text.translate(str.maketrans('', '', string.punctuation))
    #text = text.lower()
    text = [text for text in text.split() if text not in (set(sw))]
    
    for token in fw:
        if token in text:
            ret_dict.update({token: 1})
        else:
            ret_dict.update({token: 0})

    return(ret_dict)

## Confirm Vector of Features (disable for printing)

In [44]:
#conv_features("donald is the president", feature_words)

In [45]:
#assert(len(feature_words)>0)
#assert(conv_features("donald is the president",feature_words)==
#       {'donald':True,'president':True})
#assert(conv_features("people are american in america",feature_words)==
#                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [46]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [47]:
#random.seed(20220507)
random.seed(84)
random.shuffle(featuresets)

test_size = 500

In [48]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(sorted(classifier.labels()))
#print(classifier.labels())
print(nltk.classify.accuracy(classifier, test_set))

['Democratic', 'Republican']
0.736


In [49]:
print(classifier.labels())

['Republican', 'Democratic']


In [50]:
classifier.show_most_informative_features(100)

Most Informative Features
             enforcement = 1              Republ : Democr =     37.3 : 1.0
                   votes = 1              Democr : Republ =     24.6 : 1.0
                   china = 1              Republ : Democr =     19.5 : 1.0
                freedoms = 1              Republ : Democr =     17.9 : 1.0
                   media = 1              Republ : Democr =     16.0 : 1.0
                 destroy = 1              Republ : Democr =     14.8 : 1.0
                 amazing = 1              Republ : Democr =     13.8 : 1.0
              appreciate = 1              Republ : Democr =     13.8 : 1.0
                  defund = 1              Republ : Democr =     13.8 : 1.0
                supports = 1              Republ : Democr =     13.8 : 1.0
                   trade = 1              Republ : Democr =     13.3 : 1.0
                 chinese = 1              Republ : Democr =     12.8 : 1.0
                  lowest = 1              Republ : Democr =     12.8 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

Republicans are more likely to use certain words more often than Democrats.  I suspect that this is due to Republicans staying on talking points more than Democrats, as well as Republicans attempting to use similar langauge as Donald Trump to appeal to the base.  "Media, destroy, amazing, defund, trade, mike, defense" and so on are words I would expect to see used more by Republicans.  I am a bit suprised at so few words used more by Democrats in the top 20-30 except "votes".

## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [51]:
cong_db = sqlite3.connect("C:/Users/drzag/pol_class/congressional_data.db")
cong_cur = cong_db.cursor()

In [52]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

#results = list(results) # Just to store it, since the query is time consuming
#for line in results:
#    for values in line:
#        values = values.decode('utf-8')

def d_code(x):
    x = x.decode()
    return x

results_db = pd.DataFrame(results, columns = ['candidate', 'party', 'text'])
results_db = pd.DataFrame(results_db, columns = ['text', 'party'])
results_db['text'] = results_db['text'].apply(d_code)

In [53]:
results_db.tail(5)

,text,party
664651,We had a great time at the WVU Homecoming para...,Republican
664652,We need more transparency in Washington #wvpol...,Republican
664653,We saw there is a double standard in DC and th...,Republican
664654,Wow! Huge crowd in Charleston at the @WVGOP vi...,Republican
664655,https://t.co/0QmZlRfEcD https://t.co/FY520NC2GB,Republican


In [54]:
tweet_data = []

tweet_data = results_db.values.tolist()
#Apply tokenization and cleaning using Pythonic list comprehension

def custom_clean(x):
    x = [[t.lower(), p] for t, p in x]
    x = [[t.translate(str.maketrans('', '', string.punctuation)), p] for t, p in x]
    x = [[(html.unescape(t)), p] for t, p in x]   # convert html escapes like &amp; to characters.
    x = [[re.sub(r'<[^<>]*>', ' ', t), p] for t, p in x]   # tags like <tab>
    x = [[re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', t), p] for t, p in x] # markdown URLs like [Some text](https://....)
    x = [[re.sub(r'\[[^\[\]]*\]', ' ', t), p] for t, p in x] # text or code in brackets like [0]
    x = [[re.sub(r'(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)', ' ', t), p] for t, p in x] # standalone sequences of specials, matches &# but not #cool
    x = [[re.sub(r'(?:^|\s)[\-=\+]{2,}(?:\s|$)', ' ', t), p] for t, p in x] # standalone sequences of hyphens like --- or ==
    x = [[re.sub(r'\s+', ' ', t), p] for t, p in x] # sequences of white spaces
    return x

def only_words(x):
    x = [[re.sub(r'http\S+', '', t), p] for t, p in x]  # delete URLs
    x = [[re.sub('[^a-z\s]', '', t.lower()), p] for t, p in x]
    return x
    
            
# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.

In [55]:
tweet_data = custom_clean(tweet_data)

In [56]:
random.seed(42)
random.shuffle(tweet_data)

In [57]:
tweet_data_ow = only_words(tweet_data)

In [58]:
random.seed(42)
random.shuffle(tweet_data_ow)

In [59]:
word_cutoff = 50

tokens = [w for t, p in tweet_data_ow for w in t.split()]

#word_dist = nltk.FreqDist(tokens)
#for w, c in word_dist.items():
#    print(w, c)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} features in the model.")

With a word cutoff of 50, we have 417 features in the model.


In [60]:
sw = stopwords.words(["english"])
def conv_features(text,fw):
    #ret_list = []
    ret_dict = dict()
    #text = text.translate(str.maketrans('', '', string.punctuation))
    #text = text.lower()
    text = [text for text in text.split() if text not in (set(sw))]
    
    for token in fw:
        if token in text:
            ret_dict.update({token: 1})
        else:
            ret_dict.update({token: 0})

    return(ret_dict)

#conv_features("donald is the president", feature_words)

There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [61]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data_ow,k=10)
tweet_data_sample

[['angietheo will do', 'Republican'],
 ['as someone who uses the house gym i can confirm that this is the last thing anyone should be worrying about when we need to pass a responsible budgets that reauthorizes chip and protects dreamers ',
  'Democratic'],
 ['you lost the privilege of a civil conversation when you made false claims about me paying protestors harassed my campaign director and wished for more severe violence against me theres school in the morning isnt it past your bedtime ',
  'Republican'],
 ['williamsnowhume not my father', 'Democratic'],
 ['celebrating internationalwomensday with the womens leadership network at microntech ',
  'Democratic'],
 ['wall ', 'Republican'],
 ['imscotte thank you scott', 'Democratic'],
 ['we now know where  billion of drnealdunnfl s  trillion wet kiss of tax cuts for corporations and the wealthiest has gone not jobs or growth in district  ',
  'Democratic'],
 ['yes we should ', 'Republican'],
 ['this one chocolate w reeses cups all over it 

### Convert the Sample Tweets to Dictionary Features and Create Two New Lists to Iterate Over

In [62]:
tweet_data_sample_dict = [(conv_features(tweet,feature_words), party) for (tweet, party) in tweet_data_sample]
tweet_data_sample_both = [(conv_features(tweet,feature_words), tweet, party) for (tweet, party) in tweet_data_sample]

In [63]:
#random.seed(20220507)

#random.seed(84)
#random.shuffle(featuresets)

#test_size = 500

In [64]:
#test_set, train_set = featuresets[:test_size], featuresets[test_size:]
#classifier = nltk.NaiveBayesClassifier.train(train_set)
#print(sorted(classifier.labels()))
#print(nltk.classify.accuracy(classifier, test_set))

In [65]:
print(nltk.classify.accuracy(classifier, tweet_data_sample_dict))

0.6


In [66]:
for dict_tweet, party in tweet_data_sample_dict:
    print(classifier.classify(dict_tweet), party)

Democratic Republican
Democratic Democratic
Democratic Republican
Democratic Democratic
Democratic Democratic
Democratic Republican
Democratic Democratic
Democratic Democratic
Democratic Republican
Democratic Democratic


In [67]:
for dict_tweet, tweet, party in tweet_data_sample_both:
    estimated_party = classifier.classify(dict_tweet)
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: angietheo will do
Actual party is Republican and our classifer says Democratic.

Here's our (cleaned) tweet: as someone who uses the house gym i can confirm that this is the last thing anyone should be worrying about when we need to pass a responsible budgets that reauthorizes chip and protects dreamers 
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: you lost the privilege of a civil conversation when you made false claims about me paying protestors harassed my campaign director and wished for more severe violence against me theres school in the morning isnt it past your bedtime 
Actual party is Republican and our classifer says Democratic.

Here's our (cleaned) tweet: williamsnowhume not my father
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: celebrating internationalwomensday with the womens leadership network at microntech 
Actual party is Democratic and our classifer

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [68]:
# print(tweet)
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0
        print(results)

num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    #print(idx, tp)
    tweet, party = tp
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
    # get the estimated party
    dict_tweet = conv_features(tweet,feature_words)
    estimated_party = classifier.classify(dict_tweet)
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

defaultdict(<function <lambda> at 0x0000017E76F0B8B0>, {'Republican': defaultdict(<class 'int'>, {'Republican': 0})})
defaultdict(<function <lambda> at 0x0000017E76F0B8B0>, {'Republican': defaultdict(<class 'int'>, {'Republican': 0, 'Democratic': 0})})
defaultdict(<function <lambda> at 0x0000017E76F0B8B0>, {'Republican': defaultdict(<class 'int'>, {'Republican': 0, 'Democratic': 0}), 'Democratic': defaultdict(<class 'int'>, {'Republican': 0})})
defaultdict(<function <lambda> at 0x0000017E76F0B8B0>, {'Republican': defaultdict(<class 'int'>, {'Republican': 0, 'Democratic': 0}), 'Democratic': defaultdict(<class 'int'>, {'Republican': 0, 'Democratic': 0})})


In [69]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 40, 'Democratic': 4364}),
             'Democratic': defaultdict(int,
                         {'Republican': 47, 'Democratic': 5551})})

### Reflections

Using the model from classification of convention speeches appears to perform poorly when used for tweets.  The model appears to classify about 90% of all tweets as Democratic.  It seems the language used in the convention by Republicans may not represent their langauge in tweets.  We saw that there are many words used by Republicans in the convention setting far more often than Democrats, suggesting more scripted talking point discipline (or imitating Trump langauge) by Republicans.  These ratios appear not to hold up in the "real world" of Twitter.